In [47]:
from xml.dom import minidom
import pandas as pd
from unidecode import unidecode

In [31]:
inputs_path = '../inputs/'
results_path = '../results/'

## Read TXT config file

In [76]:
config_dict = {}
with open(inputs_path + 'PC.CFG', "r") as config:
    for line in config:
        print(line)
        if 'LEIA' in line:
            config_dict['read_file'] = line.split('=')[1][:-1]
        elif 'CONSULTAS' in line:
            config_dict['consult_file'] = line.split('=')[1][:-1]
        elif 'ESPERADOS' in line:
            config_dict['expect_file'] = line.split('=')[1]
    

LEIA=cfquery.xml

CONSULTAS=consultas.csv

ESPERADOS=esperados.csv


## Read XML

In [34]:
xml_file = inputs_path + config_dict['read_file']

- Remove ';' from file

In [35]:
with open(xml_file, 'r') as infile:
    data = infile.read().replace(';', '')
    
with open(xml_file, 'w') as outfile:
    outfile.write(data)

- parse xml

In [72]:
parsed_xml = minidom.parse(xml_file)
query_elements = parsed_xml.getElementsByTagName('QUERY')

- Generate Queries CSV

In [71]:
xml_data_queries = []
for query in query_elements:
    query_number = query.getElementsByTagName('QueryNumber')[0].firstChild.nodeValue
    query_text = query.getElementsByTagName('QueryText')[0].firstChild.nodeValue.strip()
    query_text = unidecode(query_text.replace('\n', '')).lower()
    xml_data_queries.append([query_number, query_text])

df_queries = pd.DataFrame(xml_data_queries, columns=['QueryNumber', 'QueryText'])

df_queries.to_csv(results_path + config_dict['consult_file'], index=False)

- Generate Expected Csv

In [77]:
# Extrai as informações do XML
xml_data_docs = []
for query in parsed_xml.getElementsByTagName('QUERY'):
    query_number = query.getElementsByTagName('QueryNumber')[0].childNodes[0].data
    for record in query.getElementsByTagName('Records')[0].getElementsByTagName('Item'):
        doc_number = record.childNodes[0].data
        doc_votes = record.getAttribute('score')
        xml_data_docs.append((query_number, doc_number, doc_votes))

# Cria um DataFrame com as informações extraídas do XML
df_docs = pd.DataFrame(xml_data_docs, columns=['QueryNumber', 'DocNumber', 'DocVotes'])
df_docs.to_csv(results_path + config_dict['expect_file'], index=False)